In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\pande\Github_Repositories\chat-bot\.venv\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

PINECONE_API_KEY = "3b198f9d-0896-4cb0-8ac2-84307fda4254"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 341


In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY)

index_name="chat-bot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [25]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "Who is the chief guest at 5 th convocation?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='with over 70 candidates qualified. On 22nd December we had the 5th Convocation of our University with Dr. Balaram \nBhargava, DG, ICMR and Head Department of Cardiology, AIIMS, New Delhi as the Chief Guest. \nDPSRU with the word ‘Research’ in the name of the university itself, has created the state-of-the-art infrastructure \nresearch facilities. With the legacy of having unique Ocular Pharmacology lab, Clinical research, Herbal Technology', metadata={}), Document(page_content='Dr. Swaminathan \nDr. Vijender Kumar \nDr. K. Praveen \nDr. Priti Jain\nDr. Varsha Chorsiya \nDr. Pooja Mongia Raj \nDr. Sakshi Bajaj\nDr. Amrita Mishra\nDr. Foziyah Zakir\nDr. Aakash Midha\nDr. Sachin\nDr. Atul Jain\nDr. Parul Sharma\nDr. Subodh Gupta\nDr. Suresh Kumar\nDr. Jitender Munjal\nMember Secretary Mr. Aditya Kaushik\nHostel Admission Mrs. Manju Vyas Singh\nDr. J. Swaminathan \nDr. Madhu Gupta\nComputer Assistant Mr. Rahul Prasad1. Vision, Mission & Objectives 2\n2. From 

In [26]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [28]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [29]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])